In [ ]:
import sys
sys.path.insert(0, "/notebooks/pipenv")
sys.path.insert(0, "/notebooks/nebula3_database")
from torch.nn.functional import softmax
from transformers import BertTokenizer, BertForNextSentencePrediction
import torch
import random
import csv
import numpy as np

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')
device = "cuda:0"
model = model.to(device)

In [ ]:
def load_stories(dataset = 'roc'):
    if dataset ==  'roc':
        stories = []
        with open("1000ROC.csv") as file:
            tsv_file = csv.reader(file, delimiter=",")
            for line in tsv_file:
                #print(line)
                line.pop(0)
                #print(line)
                line.pop(0)
                stories.append(line)
    return(stories)

In [ ]:
def create_stories(stories):
    stories_with_candidates = []
    for story in stories:
        story_length = len(orig_story) - 1
        corpus_length = len(stories) -1
        scenes = []
        #print(orig_story[0])
        for sentence in story:
            candidates = []
            candidates.append(sentence)
            for candidate in range(0, random.randint(3, 10 )):
                cand_sent = random.randint(0, story_length)
                cand_story = random.randint(0, corpus_length)
                candidates.append(stories[cand_story][cand_sent])
            scenes.append(candidates)
        stories_with_candidates.append(scenes)
    return(stories_with_candidates)

In [ ]:
for orig_story, story_with_cand in zip(stories,stories_with_candidates):
    print("ORIG: ", orig_story)
    for scene in story_with_cand: 
        print("... ",scene)

In [ ]:
def story_compatability(scene1, scene2):
    rows_ = []
    for sent_a in scene1:
        cols_ = []
        for sent_b in scene2:
            encoded = tokenizer.encode_plus(sent_a, sent_b, return_tensors='pt').to(device)
            seq_relationship_logits = model(**encoded)[0]
            probs = softmax(seq_relationship_logits, dim=1)
            score = probs[0][0].tolist()
            cols_.append(score)
        rows_.append(cols_)
    return(np.array(rows_))
    # for i in range(len(story)-1):
    #     sent_a = story[i]
    #     sent_b = story[i+1]
    #     encoded = tokenizer.encode_plus(sent_a, sent_b, return_tensors='pt').to(device)
    #     seq_relationship_logits = model(**encoded)[0]
    #     probs = softmax(seq_relationship_logits, dim=1)
    #     compt = probs[0][0].tolist()
    #     result *= compt
    # return result

In [ ]:
stories = load_stories('roc')
candidated_stories = create_stories(stories)
stories_with_scores = []
for story in candidated_stories[0:10]:
    scenes_scores = []
    for idx in range(0, len(story) -1):
        scene1 = story[idx]
        scene2 = story[idx + 1]
        scene_matrix = story_compatability(scene1, scene2)
        scenes_scores.append(scene_matrix)
    storie_with_scores = {
        'story': story,
        'scores': scenes_scores
    }

    stories_with_scores.append(storie_with_scores)

In [ ]:
for i in stories_with_scores:
    print(i)